In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [ ]:
# Apparently there is a sensor fusion DMP digital motion processor
# that should be able to process on the device 
# https://github.com/micropython-IMU/micropython-mpu9150/issues/6


In [2]:
from machine import I2C, Pin
i = I2C(scl=Pin(5, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP))
print(i.scan())

[104]


In [3]:
from machine import I2C, Pin
rst = Pin(16, Pin.OUT)
rst.value(1)

#i2c = I2C(scl=Pin(15, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP), freq=450000)
i2c = I2C(scl=Pin(5, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP))


ads = i2c.scan()
print([hex(s)  for s in ads])
if 0x68 in ads:
    print("We have an MPU6050")


['0x68']
We have an MPU6050


In [4]:
i2c.writeto_mem(0x68, 0x6B, b"\x00")  # disable sleep mode and select clock source as the XGyro


In [5]:
import ustruct
bdata = i2c.readfrom_mem(0x68, 0x3B, 14)
data = ustruct.unpack('>hhhhhhh', bdata)
accel = [x/(65536//2//2) for x in data[0:3]]
gyros = [x/(65536//250//2) for x in data[4:7]]
print(accel, gyros)


[0.3815918, 0.7937012, -0.453125] [-1.862595, -0.5267176, -1.0]


In [ ]:
import time

print(hex(i2c.readfrom_mem(0x68, 0x75, 1)[0]))  # who am I (the I2C address)

# look at MPU6050 register map datasheet
i2c.writeto_mem(0x68, 0x6B, b"\x80")  # reset device
time.sleep_ms(100)
i2c.writeto_mem(0x68, 0x68, b"\x07")  # reset gyro, accel, temp
time.sleep_ms(100)


0x68


In [ ]:
i2c.writeto_mem(0x68, 0x6B, b"\x01")  # disable sleep mode and select clock source as the XGyro
i2c.writeto_mem(0x68, 0x6C, b"\x00")  # enable all sensors (standby=0)
i2c.writeto_mem(0x68, 0x1A, b"\x01")  # Enable digital low pass filter (DLPF)
i2c.writeto_mem(0x68, 0x19, b"\x20")  # sample rate is 33Hz=1kH/(1+rate) (1kH because DLPF enabled)

accel_range = 2  # g m/s
v = i2c.readfrom_mem(0x68, 0x1C, 1)[0]
v = (v&~0x18 | 0x00)
i2c.writeto_mem(0x68, 0x1C, bytes([v]))

gyro_range = 250  # deg/sec
v = i2c.readfrom_mem(0x68, 0x1B, 1)[0]
v = (v&~0x18 | 0x00)
i2c.writeto_mem(0x68, 0x1B, bytes([v]))

import ustruct, time
bdata = bytearray(14)

In [ ]:
for i in range(10):
    i2c.readfrom_mem_into(0x68, 0x3B, bdata)
    data = ustruct.unpack('>hhhhhhh', bdata)
    accel = [x/(65536//2//2) for x in data[0:3]]
    gyros = [x/(65536//250//2) for x in data[4:7]]
    print("accel:", accel, "gyro:", gyros)
    time.sleep(1)

accel: [0.9265137, -0.01477051, 0.4494629] gyro: [-2.580153, 0.5954198, -0.8015267]
accel: [0.5335693, -0.3244629, 0.5107422] gyro: [19.49618, -37.51909, 25.07634]
accel: [0.9155273, -0.02661133, 0.4489746] gyro: [-2.458015, 0.7328244, -1.045802]
accel: [-0.2254639, -0.1268311, 1.096069] gyro: [12.03053, 250.1298, 1.839695]
accel: [-0.4547119, 0.001831055, 0.8961182] gyro: [-3.946565, -1.526718, -0.8015267]
accel: [-0.4558105, -0.006835938, 0.8970947] gyro: [-3.022901, 0.2213741, -0.8473283]
accel: [0.8946533, -0.09216309, 0.5523682] gyro: [-36.93893, -250.1374, -11.57252]
accel: [0.3571777, 0.1170654, -1.069092] gyro: [-37.58779, -36.9542, -2.916031]
accel: [0.5683594, 0.6875, -0.2215576] gyro: [-149.4275, 190.7252, 13.81679]
accel: [0.9960938, 0.3682861, -0.2001953] gyro: [-5.145038, 20.35878, 206.0076]
.

In [ ]:
# Try to get the DMP code to work
# also at https://os.mbed.com/users/paulbartell/code/MPU6050-DMP/docs/95449a48c5c0/MPU6050__6Axis__MotionApps20_8h_source.html

In [27]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [39]:
bitStart, length = 2, 3
mask = ((1 << length) - 1) << (bitStart - length + 1)
print(mask, (bitStart - length + 1))
#    i2c.writeto_mem(0x68, regAddr, bytes([b]))


7 0


In [44]:
from machine import I2C, Pin
i2c = I2C(scl=Pin(5, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP))

def writeBits(regAddr, bitStart, length, data):
    b = i2c.readfrom_mem(0x68, regAddr, 1)[0]
    mask = ((1 << length) - 1) << (bitStart - length + 1)
    b = (b & (mask ^ 0xFF)) | ((data << (bitStart - length + 1)) & mask)
    i2c.writeto_mem(0x68, regAddr, bytes([b]))

def writeBit(regAddr, bitNum, data):
    b = i2c.readfrom_mem(0x68, regAddr, 1)[0]
    b = (b | (1 << bitNum)) if (data != 0) else (b & ((1 << bitNum) ^ 0xFF))
    i2c.writeto_mem(0x68, regAddr, bytes([b]))

def writeByte(regAddr, data):
    i2c.writeto_mem(0x68, regAddr, bytes([data]))
    
# mpu.initialize();
# setClockSource() writeBits(MPU6050_RA_PWR_MGMT_1, MPU6050_PWR1_CLKSEL_BIT, MPU6050_PWR1_CLKSEL_LENGTH, MPU6050_CLOCK_PLL_XGYRO)
writeBits(0x6B, 2, 3, 0x01)
print(bin(i2c.readfrom_mem(0x68, 0x6B, 1)[0]))
        
# setFullScaleGyroRange() # writeBits(MPU6050_RA_GYRO_CONFIG, MPU6050_GCONFIG_FS_SEL_BIT, MPU6050_GCONFIG_FS_SEL_LENGTH, MPU6050_GYRO_FS_250);
writeBits(0x1B, 4, 2, 0x00)
print(bin(i2c.readfrom_mem(0x68, 0x6B, 1)[0]))

# setFullScaleAccelRange() writeBits(MPU6050_RA_ACCEL_CONFIG, MPU6050_ACONFIG_AFS_SEL_BIT, MPU6050_ACONFIG_AFS_SEL_LENGTH, MPU6050_ACCEL_FS_2)
writeBits(0x1C, 4, 2, 0x00)
print(bin(i2c.readfrom_mem(0x68, 0x6B, 1)[0]))

# setSleepEnabled(false) writeBit(MPU6050_RA_PWR_MGMT_1, MPU6050_PWR1_SLEEP_BIT, enabled)
writeBit(0x6B, 6, 0)
print(bin(i2c.readfrom_mem(0x68, 0x6B, 1)[0]))


0b1
0b1
0b1
0b1


In [43]:
i2c.writeto_mem(0x68, 0x6B, b'\x04')
print(bin(i2c.readfrom_mem(0x68, 0x6B, 1)[0]))


0b100


In [45]:
import time
# mpu.dmpInitialize();
#reset() writeBit(MPU6050_RA_PWR_MGMT_1, MPU6050_PWR1_DEVICE_RESET_BIT, true);
writeBit(0x6B, 7, 1)
time.sleep_ms(30)

# setSleepEnabled(true) writeBit(MPU6050_RA_PWR_MGMT_1, MPU6050_PWR1_SLEEP_BIT, enabled)
writeBit(0x6B, 6, 1)
print(bin(i2c.readfrom_mem(0x68, 0x6B, 1)[0]))

# setWakeCycleEnabled(true)  writeBit(MPU6050_RA_PWR_MGMT_1, MPU6050_PWR1_CYCLE_BIT, enabled)
writeBit(0x6B, 5, 1)
print(bin(i2c.readfrom_mem(0x68, 0x6B, 1)[0]))

# setSleepEnabled(false) writeBit(MPU6050_RA_PWR_MGMT_1, MPU6050_PWR1_SLEEP_BIT, enabled)
writeBit(0x6B, 6, 0)

# setMemoryBank(0x10, true, true);  # writeByte(MPU6050_RA_BANK_SEL, bank)
writeByte(0x6D, (0x10 & 0x1F)|0x20|0x40)

# setMemoryStartAddress(0x06) writeByte(MPU6050_RA_MEM_START_ADDR, address);
writeByte(0x6E, 0x06)

print("Revision @ user[16][6] = %X" % i2c.readfrom_mem(0x68, 0x6F, 1)[0])

writeByte(0x6D, (0x00 & 0x1F)) #  setMemoryBank(0, false, false);

print("OTPValid =", i2c.readfrom_mem(0x68, 0x00, 1)[0] & 0x01) # getOTPBankValid() | readBit(MPU6050_RA_XG_OFFS_TC, MPU6050_TC_OTP_BNK_VLD_BIT)


0b1000000
0b1100000
Revision @ user[16][6] = A5
OTPValid = 1


In [30]:
def readBits(regAddr, bitStart, length):
    b = i2c.readfrom_mem(0x68, regAddr, 1)[0]
    mask = ((1 << length) - 1) << (bitStart - length + 1)
    return (b & mask) >> (bitStart - length + 1)

xgOffsetTC = readBits(0x00, 6, 6)  # getXGyroOffsetTC()  readBits(MPU6050_RA_XG_OFFS_TC, MPU6050_TC_OFFSET_BIT, MPU6050_TC_OFFSET_LENGTH);
ygOffsetTC = readBits(0x01, 6, 6)    # getYGyroOffsetTC()  readBits(MPU6050_RA_YG_OFFS_TC, MPU6050_TC_OFFSET_BIT, MPU6050_TC_OFFSET_LENGTH);
zgOffsetTC = readBits(0x02, 6, 6)  # getZGyroOffsetTC()   readBits(MPU6050_RA_ZG_OFFS_TC, MPU6050_TC_OFFSET_BIT, MPU6050_TC_OFFSET_LENGTH);
print("xyzgyro offsets", xgOffsetTC, ygOffsetTC, zgOffsetTC)

writeByte(0x25, 0x7F)  # setSlaveAddress(0, 0x7F); writeByte(MPU6050_RA_I2C_SLV0_ADDR + num*3, address);

writeBit(0x6A, 5, False);  # writeBit(MPU6050_RA_USER_CTRL, MPU6050_USERCTRL_I2C_MST_EN_BIT, enabled);

writeByte(0x25, 0x68)  # setSlaveAddress(0, 0x68); writeByte(MPU6050_RA_I2C_SLV0_ADDR + num*3, address);

writeBit(0x6A, 1, True) # writeBit(MPU6050_RA_USER_CTRL, MPU6050_USERCTRL_I2C_MST_RESET_BIT, true);

time.sleep_ms(20)


xyzgyro offsets 63 62 63


In [23]:
%sendtofile --binary --source dmpMemory.bin

Sent 1929 bytes in 65 chunks to DdmpMemory.bin.


In [37]:
writeByte(0x6D, 0)  # setMemoryBank(i>>8)
writeByte(0x6E, 0) # setMemoryStartAddress(address)     writeByte(MPU6050_RA_MEM_START_ADDR, address);
x = i2c.writeto_mem(0x68, 0x6F, b"ABCDEFGHIJKLMNOP")

writeByte(0x6D, 0)  # setMemoryBank(i>>8)
writeByte(0x6E, 0) # setMemoryStartAddress(address)     writeByte(MPU6050_RA_MEM_START_ADDR, address);
x = i2c.readfrom_mem(0x68, 0x6F, 16)
print(x)

print(hex(i2c.readfrom_mem(0x68, 0x75, 1)[0]))  # who am I (the I2C address)


b'\x00\x00\x00\x00\x00\x00h\x00\x00\x00\x00\x00\x00\x00\x00\x00'
0x68


In [31]:
#writeProgMemoryBlock
fin = open("dmpMemory.bin", "rb")
i = 0
while True:
    data = fin.read(16)
    if len(data) == 0:
        break
    writeByte(0x6D, ((i>>8) & 0x1F))  # setMemoryBank(i>>8)
    writeByte(0x6E, (i&0xFF)) # setMemoryStartAddress(address)     writeByte(MPU6050_RA_MEM_START_ADDR, address);
    i2c.writeto_mem(0x68, 0x6F, data)  # MPU6050_RA_MEM_R_W
    i += len(data)
assert i == 1929

# check first and last sets of data are good
writeByte(0x6D, 0)  # setMemoryBank(i>>8)
writeByte(0x6E, 0) # setMemoryStartAddress(address)     writeByte(MPU6050_RA_MEM_START_ADDR, address);
x = i2c.readfrom_mem(0x68, 0x6F, 16)
assert x == b'\xfb\x00\x00>\x00\x0b\x006\x00\x01\x00\x02\x00\x03\x00\x00', x

writeByte(0x6D, 1929//256)
writeByte(0x6E, ((1929%256)//16)*16)
x = i2c.readfrom_mem(0x68, 0x6F, 1929%16)
assert x == b'\xb9\xa7\xf1&&&\xd8\xd8\xff', x


Traceback (most recent call last):
  File "<stdin>", line 18, in <module>
AssertionError: b'\xff\x00\x00\x00\x00\xffh\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [49]:
%sendtofile dmpConfig.txt
    0x02,   0xEC,   0x04,   0x00, 0x47, 0x7D, 0x1A,   // ?
    0x03,   0x82,   0x03,   0x4C, 0xCD, 0x6C,         // FCFG_1 inv_set_gyro_calibration
    0x03,   0xB2,   0x03,   0x36, 0x56, 0x76,         // FCFG_3 inv_set_gyro_calibration
    0x00,   0x68,   0x04,   0x02, 0xCA, 0xE3, 0x09,   // D_0_104 inv_set_gyro_calibration
    0x01,   0x0C,   0x04,   0x00, 0x00, 0x00, 0x00,   // D_1_152 inv_set_accel_calibration
    0x03,   0x86,   0x03,   0x0C, 0xC9, 0x2C,         // FCFG_2 inv_set_accel_calibration
    0x03,   0x90,   0x03,   0x26, 0x46, 0x66,         //   (continued)...FCFG_2 inv_set_accel_calibration
    0x00,   0x6C,   0x02,   0x40, 0x00,               // D_0_108 inv_set_accel_calibration

    0x02,   0x40,   0x04,   0x00, 0x00, 0x00, 0x00,   // CPASS_MTX_00 inv_set_compass_calibration
    0x02,   0x44,   0x04,   0x40, 0x00, 0x00, 0x00,   // CPASS_MTX_01
    0x02,   0x48,   0x04,   0x00, 0x00, 0x00, 0x00,   // CPASS_MTX_02
    0x02,   0x4C,   0x04,   0x40, 0x00, 0x00, 0x00,   // CPASS_MTX_10
    0x02,   0x50,   0x04,   0x00, 0x00, 0x00, 0x00,   // CPASS_MTX_11
    0x02,   0x54,   0x04,   0x00, 0x00, 0x00, 0x00,   // CPASS_MTX_12
    0x02,   0x58,   0x04,   0x00, 0x00, 0x00, 0x00,   // CPASS_MTX_20
    0x02,   0x5C,   0x04,   0x00, 0x00, 0x00, 0x00,   // CPASS_MTX_21
    0x02,   0xBC,   0x04,   0xC0, 0x00, 0x00, 0x00,   // CPASS_MTX_22

    0x01,   0xEC,   0x04,   0x00, 0x00, 0x40, 0x00,   // D_1_236 inv_apply_endian_accel
    0x03,   0x86,   0x06,   0x0C, 0xC9, 0x2C, 0x97, 0x97, 0x97, // FCFG_2 inv_set_mpu_sensors
    0x04,   0x22,   0x03,   0x0D, 0x35, 0x5D,         // CFG_MOTION_BIAS inv_turn_on_bias_from_no_motion
    0x00,   0xA3,   0x01,   0x00,                     // ?
    0x04,   0x29,   0x04,   0x87, 0x2D, 0x35, 0x3D,   // FCFG_5 inv_set_bias_update
    0x07,   0x62,   0x05,   0xF1, 0x20, 0x28, 0x30, 0x38, // CFG_8 inv_send_quaternion
    0x07,   0x9F,   0x01,   0x30,                     // CFG_16 inv_set_footer
    0x07,   0x67,   0x01,   0x9A,                     // CFG_GYRO_SOURCE inv_send_gyro
    0x07,   0x68,   0x04,   0xF1, 0x28, 0x30, 0x38,   // CFG_9 inv_send_gyro -> inv_construct3_fifo
    0x07,   0x62,   0x05,   0xF1, 0x20, 0x28, 0x30, 0x38, // ?
    0x02,   0x0C,   0x04,   0x00, 0x00, 0x00, 0x00,   // ?
    0x07,   0x83,   0x06,   0xC2, 0xCA, 0xC4, 0xA3, 0xA3, 0xA3, // ?
                 // SPECIAL 0x01 = enable interrupts
    0x00,   0x00,   0x00,   0x01, // SET INT_ENABLE, SPECIAL INSTRUCTION
    0x07,   0xA7,   0x01,   0xFE,                     // ?
    0x07,   0x62,   0x05,   0xF1, 0x20, 0x28, 0x30, 0x38, // ?
    0x07,   0x67,   0x01,   0x9A,                     // ?
    0x07,   0x68,   0x04,   0xF1, 0x28, 0x30, 0x38,   // CFG_12 inv_send_accel -> inv_construct3_fifo
    0x07,   0x8D,   0x04,   0xF1, 0x28, 0x30, 0x38,   // ??? CFG_12 inv_send_mag -> inv_construct3_fifo
    0x02,   0x16,   0x02,   0x00, 0x03                // D_0_22 inv_set_fifo_rate

    // This very last 0x01 WAS a 0x09, which drops the FIFO rate down to 20 Hz. 0x07 is 25 Hz,
    // 0x01 is 100Hz. Going faster than 100Hz (0x00=200Hz) tends to result in very noisy data.
    // DMP output frequency is calculated easily using this equation: (200Hz / (1 + value))

    // It is important to make sure the host processor can keep up with reading and processing
    // the FIFO output at the desired rate. Handling FIFO overflow cleanly is also a good idea.


Sent 46 lines (3409 bytes) to dmpConfig.txt.


In [34]:
for l in open("dmpConfig.txt"):
    v = l.split("/", 1)[0].strip()
    if not v:`
        continue
    v = [int(k.strip()[2:4], 16)  for k in v.split(",")  if k]
    bank, offset, length = v[:3]
    if length > 0:
        assert length == len(v[3:])
        writeByte(0x6D, bank)   # setMemoryBank(i>>8)
        writeByte(0x6E, offset) # setMemoryStartAddress(address)     writeByte(MPU6050_RA_MEM_START_ADDR, address);
        i2c.writeto_mem(0x68, 0x6F, bytes(v[3:]))
    else:
        special = v[3]
        assert special == 0x01
        writeByte(0x38, 0x32)   # writeByte(MPU6050_RA_INT_ENABLE, 0x32)

writeByte(0x6D, 0x07)  # setMemoryBank(i>>8)
writeByte(0x6E, 0x8D) # setMemoryStartAddress(address)     writeByte(MPU6050_RA_MEM_START_ADDR, address);
x = i2c.readfrom_mem(0x68, 0x6F, 4)
assert x == b'\xf1(08', x


Traceback (most recent call last):
  File "<stdin>", line 20, in <module>
AssertionError: b'\xff\x00\x00\x00'


In [ ]:
writeBits(0x6B, 2, 3, 0x03) # setClockSource(MPU6050_CLOCK_PLL_ZGYRO)     writeBits(MPU6050_RA_PWR_MGMT_1, MPU6050_PWR1_CLKSEL_BIT, MPU6050_PWR1_CLKSEL_LENGTH, source)

writeByte(0x38, 0x12) # setIntEnabled(0x12) writeByte(MPU6050_RA_INT_ENABLE, enabled)

writeByte(0x19, 4) # setRate(4); // 1khz / (1 + 4) = 200 Hz  writeByte(MPU6050_RA_SMPLRT_DIV, rate);

writeBits(0x1A, 5, 3, 0x01) # setExternalFrameSync(MPU6050_EXT_SYNC_TEMP_OUT_L)  writeBits(MPU6050_RA_CONFIG, MPU6050_CFG_EXT_SYNC_SET_BIT, MPU6050_CFG_EXT_SYNC_SET_LENGTH, sync);

writeBits(0x1A, 2, 3, 0x03) # setDLPFMode(MPU6050_DLPF_BW_42); #     writeBits(MPU6050_RA_CONFIG, MPU6050_CFG_DLPF_CFG_BIT, MPU6050_CFG_DLPF_CFG_LENGTH, mode);

writeBits(0x1B, 4, 2, 0x03) # setFullScaleGyroRange(MPU6050_GYRO_FS_2000); writeBits(MPU6050_RA_GYRO_CONFIG, MPU6050_GCONFIG_FS_SEL_BIT, MPU6050_GCONFIG_FS_SEL_LENGTH, range);

writeByte(0x70, 0x03) # setDMPConfig1(0x03);     writeByte(MPU6050_RA_DMP_CFG_1, config);

writeByte(0x71, 0x00) # setDMPConfig2(0x00); writeByte(MPU6050_RA_DMP_CFG_2, config);

writeBit(0x00, 0, False)  # setOTPBankValid(false); writeBit(MPU6050_RA_XG_OFFS_TC, MPU6050_TC_OTP_BNK_VLD_BIT, enabled);

xgOffsetTC, ygOffsetTC, zgOffsetTC = 63, 62, 63
writeBits(0x00, 6, 6, xgOffsetTC)  # setXGyroOffsetTC(xgOffsetTC)  writeBits(MPU6050_RA_XG_OFFS_TC, MPU6050_TC_OFFSET_BIT, MPU6050_TC_OFFSET_LENGTH, offset);
writeBits(0x01, 6, 6, ygOffsetTC)  # setYGyroOffsetTC(ygOffsetTC)  writeBits(MPU6050_RA_YG_OFFS_TC, MPU6050_TC_OFFSET_BIT, MPU6050_TC_OFFSET_LENGTH, offset);
writeBits(0x02, 6, 6, ygOffsetTC)  # setYGyroOffsetTC(zgOffsetTC)  writeBits(MPU6050_RA_ZG_OFFS_TC, MPU6050_TC_OFFSET_BIT, MPU6050_TC_OFFSET_LENGTH, offset);


In [ ]:
# lines from dmpUpdates
writeByte(0x6D, 0x01);  writeByte(0x6E, 0xB2);  i2c.writeto_mem(0x68, 0x6F, b"\xFF\xFF") # dmpUpdates1
writeByte(0x6D, 0x01);  writeByte(0x6E, 0x90);  i2c.writeto_mem(0x68, 0x6F, b"\x09\x23\xA1\x35") # dmpUpdates2

writeBit(0x6A, 2, True) # resetFIFO() # writeBit(MPU6050_RA_USER_CTRL, MPU6050_USERCTRL_FIFO_RESET_BIT, true);

def getFIFOCount():
    b = i2c.readfrom_mem(0x68, 0x72, 2)   # getFIFOCount() # readBytes(devAddr, MPU6050_RA_FIFO_COUNTH, 2, buffer);
    return (b[0]<<8) + b[1]
fifoCount = getFIFOCount()

fifoBuffer = i2c.readfrom_mem(0x68, 0x74, fifoCount) # getFIFOBytes(fifoBuffer, fifoCount); readBytes(MPU6050_RA_FIFO_R_W, length, data);

writeByte(0x1F, 2) # setMotionDetectionThreshold(2);    writeByte(MPU6050_RA_MOT_THR, threshold);

writeByte(0x21, 156) # setZeroMotionDetectionThreshold(156); # writeByte(MPU6050_RA_ZRMOT_THR, threshold);

writeByte(0x20, 80) # setMotionDetectionDuration(80);     writeByte(MPU6050_RA_MOT_DUR, duration);

writeByte(0x22, 0) # setZeroMotionDetectionDuration(0); writeByte(MPU6050_RA_ZRMOT_DUR, duration);

writeBit(0x6A, 2, True) # resetFIFO() # writeBit(MPU6050_RA_USER_CTRL, MPU6050_USERCTRL_FIFO_RESET_BIT, true);

writeBit(0x6A, 6, True) # setFIFOEnabled(true);     writeBit(MPU6050_RA_USER_CTRL, MPU6050_USERCTRL_FIFO_EN_BIT, enabled);

writeBit(0x6A, 7, True) # setDMPEnabled(true);     writeBit(MPU6050_RA_USER_CTRL, MPU6050_USERCTRL_DMP_EN_BIT, enabled);

writeBit(0x6A, 3, True) # resetDMP();     writeBit(MPU6050_RA_USER_CTRL, MPU6050_USERCTRL_DMP_RESET_BIT, true);

writeByte(0x6D, 0x01);  writeByte(0x6E, 0x6A);  i2c.writeto_mem(0x68, 0x6F, b"\x06\x00") # dmpUpdates3
writeByte(0x6D, 0x01);  writeByte(0x6E, 0x60);  i2c.writeto_mem(0x68, 0x6F, b"\x00\x00\x00\x00\x00\x00\x00\x00") # dmpUpdates4
writeByte(0x6D, 0x00);  writeByte(0x6E, 0x60);  i2c.writeto_mem(0x68, 0x6F, b"\x40\x00\x00\x00") # dmpUpdates5


while True:
    fifoCount = getFIFOCount()
    if fifoCount >= 3:  break
print("fifoCount", fifoCount)
fifoBuffer = i2c.readfrom_mem(0x68, 0x74, fifoCount)

print("getIntStatus() ", i2c.readfrom_mem(0x68, 0x3A, 1)[0])

writeByte(0x6D, 0x01);  writeByte(0x6E, 0x62);  i2c.writeto_mem(0x68, 0x6F, b"\x00\x00") # dmpUpdates6
            
while True:
    fifoCount = getFIFOCount()
    if fifoCount >= 3:  break
print("fifoCount", fifoCount)
fifoBuffer = i2c.readfrom_mem(0x68, 0x74, fifoCount)

print("getIntStatus() ", i2c.readfrom_mem(0x68, 0x3A, 1)[0])

writeByte(0x6D, 0x00);  writeByte(0x6E, 0x60);  i2c.writeto_mem(0x68, 0x6F, b"\x00\x40\x00\x00") # dmpUpdates7




In [ ]:

# DEBUG_PRINTLN(F("Disabling DMP (you turn it on later)..."));
writeBit(0x6A, 7, False) # setDMPEnabled(False);     writeBit(MPU6050_RA_USER_CTRL, MPU6050_USERCTRL_DMP_EN_BIT, enabled);

dmpPacketSize = 42;

DEBUG_PRINTLN(F("Resetting FIFO and clearing INT status one last time..."));
writeBit(0x6A, 2, True) # resetFIFO() # writeBit(MPU6050_RA_USER_CTRL, MPU6050_USERCTRL_FIFO_RESET_BIT, true);
print("getIntStatus() ", i2c.readfrom_mem(0x68, 0x3A, 1)[0])

#mpu.setXGyroOffset(220);
#mpu.setYGyroOffset(76);
#mpu.setZGyroOffset(-85);
#mpu.setZAccelOffset(17True);

writeBit(0x6A, 7, True) # setDMPEnabled(True);     writeBit(MPU6050_RA_USER_CTRL, MPU6050_USERCTRL_DMP_EN_BIT, enabled);


In [ ]:


	while(1){
	    mpuIntStatus = mpu.getIntStatus();
		// get current FIFO count
		fifoCount = mpu.getFIFOCount();

	    if ((mpuIntStatus & 0x10) || fifoCount == 1024) {
	        // reset so we can continue cleanly
	        mpu.resetFIFO();

	    // otherwise, check for DMP data ready interrupt frequently)
	    } else if (mpuIntStatus & 0x02) {
	        // wait for correct available data length, should be a VERY short wait
	        while (fifoCount < packetSize) fifoCount = mpu.getFIFOCount();

	        // read a packet from FIFO

	        mpu.getFIFOBytes(fifoBuffer, packetSize);
	 		mpu.dmpGetQuaternion(&q, fifoBuffer);
			mpu.dmpGetGravity(&gravity, &q);
			mpu.dmpGetYawPitchRoll(ypr, &q, &gravity);
			printf("YAW: %3.1f, ", ypr[0] * 180/M_PI);
			printf("PITCH: %3.1f, ", ypr[1] * 180/M_PI);
			printf("ROLL: %3.1f \n", ypr[2] * 180/M_PI);
	    }

	    //Best result is to match with DMP refresh rate
	    // Its last value in components/MPU6050/MPU6050_6Axis_MotionApps20.h file line 310
	    // Now its 0x13, which means DMP is refreshed with 10Hz rate
		vTaskDelay(100/portTICK_PERIOD_MS);
	}


In [3]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Traceback (most recent call last):
  File "main.py", line 10, in <module>
  File "fmqttqueue.py", line 1, in <module>
  File "mqtt_as.py", line 28, in <module>
KeyboardInterrupt: 
MicroPython v1.10-227-g696549d2e on 2019-03-17; ESP32 module with ESP32
Type "help()" for more information.
>>> 
>>> 
raw REPL; CTRL-B to exit
>Ready.


In [100]:
import ubinascii
print(ubinascii.unhexlify("ABBCC"))


Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
ValueError: odd-length string


In [96]:
# create the file with the DMP code in it (doesn't matter the efficiency as just to create it)
fout = open("dmpMemory.bin", "wb")
for b in [
    0xFB, 0x00, 0x00, 0x3E, 0x00, 0x0B, 0x00, 0x36, 0x00, 0x01, 0x00, 0x02, 0x00, 0x03, 0x00, 0x00,
    0x00, 0x65, 0x00, 0x54, 0xFF, 0xEF, 0x00, 0x00, 0xFA, 0x80, 0x00, 0x0B, 0x12, 0x82, 0x00, 0x01,
    0x00, 0x02, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x28, 0x00, 0x00, 0xFF, 0xFF, 0x45, 0x81, 0xFF, 0xFF, 0xFA, 0x72, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x03, 0xE8, 0x00, 0x00, 0x00, 0x01, 0x00, 0x01, 0x7F, 0xFF, 0xFF, 0xFE, 0x80, 0x01,
    0x00, 0x1B, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x3E, 0x03, 0x30, 0x40, 0x00, 0x00, 0x00, 0x02, 0xCA, 0xE3, 0x09, 0x3E, 0x80, 0x00, 0x00,
    0x20, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x40, 0x00, 0x00, 0x00, 0x60, 0x00, 0x00, 0x00,
    0x41, 0xFF, 0x00, 0x00, 0x00, 0x00, 0x0B, 0x2A, 0x00, 0x00, 0x16, 0x55, 0x00, 0x00, 0x21, 0x82,
    0xFD, 0x87, 0x26, 0x50, 0xFD, 0x80, 0x00, 0x00, 0x00, 0x1F, 0x00, 0x00, 0x00, 0x05, 0x80, 0x00,
    0x00, 0x00, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00,
    0x40, 0x00, 0x00, 0x00, 0x00, 0x00, 0x04, 0x6F, 0x00, 0x02, 0x65, 0x32, 0x00, 0x00, 0x5E, 0xC0,
    0x40, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0xFB, 0x8C, 0x6F, 0x5D, 0xFD, 0x5D, 0x08, 0xD9, 0x00, 0x7C, 0x73, 0x3B, 0x00, 0x6C, 0x12, 0xCC,
    0x32, 0x00, 0x13, 0x9D, 0x32, 0x00, 0xD0, 0xD6, 0x32, 0x00, 0x08, 0x00, 0x40, 0x00, 0x01, 0xF4,
    0xFF, 0xE6, 0x80, 0x79, 0x02, 0x00, 0x00, 0x00, 0x00, 0x00, 0xD0, 0xD6, 0x00, 0x00, 0x27, 0x10,

    0xFB, 0x00, 0x00, 0x00, 0x40, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x01, 0x00, 0x01, 0x00, 0x00, 0x00,
    0x00, 0x00, 0xFA, 0x36, 0xFF, 0xBC, 0x30, 0x8E, 0x00, 0x05, 0xFB, 0xF0, 0xFF, 0xD9, 0x5B, 0xC8,
    0xFF, 0xD0, 0x9A, 0xBE, 0x00, 0x00, 0x10, 0xA9, 0xFF, 0xF4, 0x1E, 0xB2, 0x00, 0xCE, 0xBB, 0xF7,
    0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x02, 0x00, 0x02, 0x02, 0x00, 0x00, 0x0C,
    0xFF, 0xC2, 0x80, 0x00, 0x00, 0x01, 0x80, 0x00, 0x00, 0xCF, 0x80, 0x00, 0x40, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x06, 0x00, 0x00, 0x00, 0x00, 0x14,
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x00, 0x03, 0x3F, 0x68, 0xB6, 0x79, 0x35, 0x28, 0xBC, 0xC6, 0x7E, 0xD1, 0x6C,
    0x80, 0x00, 0x00, 0x00, 0x40, 0x00, 0x00, 0x00, 0x00, 0x00, 0xB2, 0x6A, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x3F, 0xF0, 0x00, 0x00, 0x00, 0x30,
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x25, 0x4D, 0x00, 0x2F, 0x70, 0x6D, 0x00, 0x00, 0x05, 0xAE, 0x00, 0x0C, 0x02, 0xD0,

    0x00, 0x00, 0x00, 0x00, 0x00, 0x65, 0x00, 0x54, 0xFF, 0xEF, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x01, 0x00, 0x00, 0x44, 0x00, 0x00, 0x00, 0x00, 0x0C, 0x00, 0x00, 0x00, 0x01, 0x00,
    0x00, 0x00, 0x00, 0x00, 0x00, 0x65, 0x00, 0x00, 0x00, 0x54, 0x00, 0x00, 0xFF, 0xEF, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x40, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x40, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x02, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x1B, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x40, 0x00, 0x00, 0x00,
    0x00, 0x1B, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,

    0xD8, 0xDC, 0xBA, 0xA2, 0xF1, 0xDE, 0xB2, 0xB8, 0xB4, 0xA8, 0x81, 0x91, 0xF7, 0x4A, 0x90, 0x7F,
    0x91, 0x6A, 0xF3, 0xF9, 0xDB, 0xA8, 0xF9, 0xB0, 0xBA, 0xA0, 0x80, 0xF2, 0xCE, 0x81, 0xF3, 0xC2,
    0xF1, 0xC1, 0xF2, 0xC3, 0xF3, 0xCC, 0xA2, 0xB2, 0x80, 0xF1, 0xC6, 0xD8, 0x80, 0xBA, 0xA7, 0xDF,
    0xDF, 0xDF, 0xF2, 0xA7, 0xC3, 0xCB, 0xC5, 0xB6, 0xF0, 0x87, 0xA2, 0x94, 0x24, 0x48, 0x70, 0x3C,
    0x95, 0x40, 0x68, 0x34, 0x58, 0x9B, 0x78, 0xA2, 0xF1, 0x83, 0x92, 0x2D, 0x55, 0x7D, 0xD8, 0xB1,
    0xB4, 0xB8, 0xA1, 0xD0, 0x91, 0x80, 0xF2, 0x70, 0xF3, 0x70, 0xF2, 0x7C, 0x80, 0xA8, 0xF1, 0x01,
    0xB0, 0x98, 0x87, 0xD9, 0x43, 0xD8, 0x86, 0xC9, 0x88, 0xBA, 0xA1, 0xF2, 0x0E, 0xB8, 0x97, 0x80,
    0xF1, 0xA9, 0xDF, 0xDF, 0xDF, 0xAA, 0xDF, 0xDF, 0xDF, 0xF2, 0xAA, 0xC5, 0xCD, 0xC7, 0xA9, 0x0C,
    0xC9, 0x2C, 0x97, 0x97, 0x97, 0x97, 0xF1, 0xA9, 0x89, 0x26, 0x46, 0x66, 0xB0, 0xB4, 0xBA, 0x80,
    0xAC, 0xDE, 0xF2, 0xCA, 0xF1, 0xB2, 0x8C, 0x02, 0xA9, 0xB6, 0x98, 0x00, 0x89, 0x0E, 0x16, 0x1E,
    0xB8, 0xA9, 0xB4, 0x99, 0x2C, 0x54, 0x7C, 0xB0, 0x8A, 0xA8, 0x96, 0x36, 0x56, 0x76, 0xF1, 0xB9,
    0xAF, 0xB4, 0xB0, 0x83, 0xC0, 0xB8, 0xA8, 0x97, 0x11, 0xB1, 0x8F, 0x98, 0xB9, 0xAF, 0xF0, 0x24,
    0x08, 0x44, 0x10, 0x64, 0x18, 0xF1, 0xA3, 0x29, 0x55, 0x7D, 0xAF, 0x83, 0xB5, 0x93, 0xAF, 0xF0,
    0x00, 0x28, 0x50, 0xF1, 0xA3, 0x86, 0x9F, 0x61, 0xA6, 0xDA, 0xDE, 0xDF, 0xD9, 0xFA, 0xA3, 0x86,
    0x96, 0xDB, 0x31, 0xA6, 0xD9, 0xF8, 0xDF, 0xBA, 0xA6, 0x8F, 0xC2, 0xC5, 0xC7, 0xB2, 0x8C, 0xC1,
    0xB8, 0xA2, 0xDF, 0xDF, 0xDF, 0xA3, 0xDF, 0xDF, 0xDF, 0xD8, 0xD8, 0xF1, 0xB8, 0xA8, 0xB2, 0x86,

    0xB4, 0x98, 0x0D, 0x35, 0x5D, 0xB8, 0xAA, 0x98, 0xB0, 0x87, 0x2D, 0x35, 0x3D, 0xB2, 0xB6, 0xBA,
    0xAF, 0x8C, 0x96, 0x19, 0x8F, 0x9F, 0xA7, 0x0E, 0x16, 0x1E, 0xB4, 0x9A, 0xB8, 0xAA, 0x87, 0x2C,
    0x54, 0x7C, 0xB9, 0xA3, 0xDE, 0xDF, 0xDF, 0xA3, 0xB1, 0x80, 0xF2, 0xC4, 0xCD, 0xC9, 0xF1, 0xB8,
    0xA9, 0xB4, 0x99, 0x83, 0x0D, 0x35, 0x5D, 0x89, 0xB9, 0xA3, 0x2D, 0x55, 0x7D, 0xB5, 0x93, 0xA3,
    0x0E, 0x16, 0x1E, 0xA9, 0x2C, 0x54, 0x7C, 0xB8, 0xB4, 0xB0, 0xF1, 0x97, 0x83, 0xA8, 0x11, 0x84,
    0xA5, 0x09, 0x98, 0xA3, 0x83, 0xF0, 0xDA, 0x24, 0x08, 0x44, 0x10, 0x64, 0x18, 0xD8, 0xF1, 0xA5,
    0x29, 0x55, 0x7D, 0xA5, 0x85, 0x95, 0x02, 0x1A, 0x2E, 0x3A, 0x56, 0x5A, 0x40, 0x48, 0xF9, 0xF3,
    0xA3, 0xD9, 0xF8, 0xF0, 0x98, 0x83, 0x24, 0x08, 0x44, 0x10, 0x64, 0x18, 0x97, 0x82, 0xA8, 0xF1,
    0x11, 0xF0, 0x98, 0xA2, 0x24, 0x08, 0x44, 0x10, 0x64, 0x18, 0xDA, 0xF3, 0xDE, 0xD8, 0x83, 0xA5,
    0x94, 0x01, 0xD9, 0xA3, 0x02, 0xF1, 0xA2, 0xC3, 0xC5, 0xC7, 0xD8, 0xF1, 0x84, 0x92, 0xA2, 0x4D,
    0xDA, 0x2A, 0xD8, 0x48, 0x69, 0xD9, 0x2A, 0xD8, 0x68, 0x55, 0xDA, 0x32, 0xD8, 0x50, 0x71, 0xD9,
    0x32, 0xD8, 0x70, 0x5D, 0xDA, 0x3A, 0xD8, 0x58, 0x79, 0xD9, 0x3A, 0xD8, 0x78, 0x93, 0xA3, 0x4D,
    0xDA, 0x2A, 0xD8, 0x48, 0x69, 0xD9, 0x2A, 0xD8, 0x68, 0x55, 0xDA, 0x32, 0xD8, 0x50, 0x71, 0xD9,
    0x32, 0xD8, 0x70, 0x5D, 0xDA, 0x3A, 0xD8, 0x58, 0x79, 0xD9, 0x3A, 0xD8, 0x78, 0xA8, 0x8A, 0x9A,
    0xF0, 0x28, 0x50, 0x78, 0x9E, 0xF3, 0x88, 0x18, 0xF1, 0x9F, 0x1D, 0x98, 0xA8, 0xD9, 0x08, 0xD8,
    0xC8, 0x9F, 0x12, 0x9E, 0xF3, 0x15, 0xA8, 0xDA, 0x12, 0x10, 0xD8, 0xF1, 0xAF, 0xC8, 0x97, 0x87,

    0x34, 0xB5, 0xB9, 0x94, 0xA4, 0x21, 0xF3, 0xD9, 0x22, 0xD8, 0xF2, 0x2D, 0xF3, 0xD9, 0x2A, 0xD8,
    0xF2, 0x35, 0xF3, 0xD9, 0x32, 0xD8, 0x81, 0xA4, 0x60, 0x60, 0x61, 0xD9, 0x61, 0xD8, 0x6C, 0x68,
    0x69, 0xD9, 0x69, 0xD8, 0x74, 0x70, 0x71, 0xD9, 0x71, 0xD8, 0xB1, 0xA3, 0x84, 0x19, 0x3D, 0x5D,
    0xA3, 0x83, 0x1A, 0x3E, 0x5E, 0x93, 0x10, 0x30, 0x81, 0x10, 0x11, 0xB8, 0xB0, 0xAF, 0x8F, 0x94,
    0xF2, 0xDA, 0x3E, 0xD8, 0xB4, 0x9A, 0xA8, 0x87, 0x29, 0xDA, 0xF8, 0xD8, 0x87, 0x9A, 0x35, 0xDA,
    0xF8, 0xD8, 0x87, 0x9A, 0x3D, 0xDA, 0xF8, 0xD8, 0xB1, 0xB9, 0xA4, 0x98, 0x85, 0x02, 0x2E, 0x56,
    0xA5, 0x81, 0x00, 0x0C, 0x14, 0xA3, 0x97, 0xB0, 0x8A, 0xF1, 0x2D, 0xD9, 0x28, 0xD8, 0x4D, 0xD9,
    0x48, 0xD8, 0x6D, 0xD9, 0x68, 0xD8, 0xB1, 0x84, 0x0D, 0xDA, 0x0E, 0xD8, 0xA3, 0x29, 0x83, 0xDA,
    0x2C, 0x0E, 0xD8, 0xA3, 0x84, 0x49, 0x83, 0xDA, 0x2C, 0x4C, 0x0E, 0xD8, 0xB8, 0xB0, 0xA8, 0x8A,
    0x9A, 0xF5, 0x20, 0xAA, 0xDA, 0xDF, 0xD8, 0xA8, 0x40, 0xAA, 0xD0, 0xDA, 0xDE, 0xD8, 0xA8, 0x60,
    0xAA, 0xDA, 0xD0, 0xDF, 0xD8, 0xF1, 0x97, 0x86, 0xA8, 0x31, 0x9B, 0x06, 0x99, 0x07, 0xAB, 0x97,
    0x28, 0x88, 0x9B, 0xF0, 0x0C, 0x20, 0x14, 0x40, 0xB8, 0xB0, 0xB4, 0xA8, 0x8C, 0x9C, 0xF0, 0x04,
    0x28, 0x51, 0x79, 0x1D, 0x30, 0x14, 0x38, 0xB2, 0x82, 0xAB, 0xD0, 0x98, 0x2C, 0x50, 0x50, 0x78,
    0x78, 0x9B, 0xF1, 0x1A, 0xB0, 0xF0, 0x8A, 0x9C, 0xA8, 0x29, 0x51, 0x79, 0x8B, 0x29, 0x51, 0x79,
    0x8A, 0x24, 0x70, 0x59, 0x8B, 0x20, 0x58, 0x71, 0x8A, 0x44, 0x69, 0x38, 0x8B, 0x39, 0x40, 0x68,
    0x8A, 0x64, 0x48, 0x31, 0x8B, 0x30, 0x49, 0x60, 0xA5, 0x88, 0x20, 0x09, 0x71, 0x58, 0x44, 0x68,

    0x11, 0x39, 0x64, 0x49, 0x30, 0x19, 0xF1, 0xAC, 0x00, 0x2C, 0x54, 0x7C, 0xF0, 0x8C, 0xA8, 0x04,
    0x28, 0x50, 0x78, 0xF1, 0x88, 0x97, 0x26, 0xA8, 0x59, 0x98, 0xAC, 0x8C, 0x02, 0x26, 0x46, 0x66,
    0xF0, 0x89, 0x9C, 0xA8, 0x29, 0x51, 0x79, 0x24, 0x70, 0x59, 0x44, 0x69, 0x38, 0x64, 0x48, 0x31,
    0xA9, 0x88, 0x09, 0x20, 0x59, 0x70, 0xAB, 0x11, 0x38, 0x40, 0x69, 0xA8, 0x19, 0x31, 0x48, 0x60,
    0x8C, 0xA8, 0x3C, 0x41, 0x5C, 0x20, 0x7C, 0x00, 0xF1, 0x87, 0x98, 0x19, 0x86, 0xA8, 0x6E, 0x76,
    0x7E, 0xA9, 0x99, 0x88, 0x2D, 0x55, 0x7D, 0x9E, 0xB9, 0xA3, 0x8A, 0x22, 0x8A, 0x6E, 0x8A, 0x56,
    0x8A, 0x5E, 0x9F, 0xB1, 0x83, 0x06, 0x26, 0x46, 0x66, 0x0E, 0x2E, 0x4E, 0x6E, 0x9D, 0xB8, 0xAD,
    0x00, 0x2C, 0x54, 0x7C, 0xF2, 0xB1, 0x8C, 0xB4, 0x99, 0xB9, 0xA3, 0x2D, 0x55, 0x7D, 0x81, 0x91,
    0xAC, 0x38, 0xAD, 0x3A, 0xB5, 0x83, 0x91, 0xAC, 0x2D, 0xD9, 0x28, 0xD8, 0x4D, 0xD9, 0x48, 0xD8,
    0x6D, 0xD9, 0x68, 0xD8, 0x8C, 0x9D, 0xAE, 0x29, 0xD9, 0x04, 0xAE, 0xD8, 0x51, 0xD9, 0x04, 0xAE,
    0xD8, 0x79, 0xD9, 0x04, 0xD8, 0x81, 0xF3, 0x9D, 0xAD, 0x00, 0x8D, 0xAE, 0x19, 0x81, 0xAD, 0xD9,
    0x01, 0xD8, 0xF2, 0xAE, 0xDA, 0x26, 0xD8, 0x8E, 0x91, 0x29, 0x83, 0xA7, 0xD9, 0xAD, 0xAD, 0xAD,
    0xAD, 0xF3, 0x2A, 0xD8, 0xD8, 0xF1, 0xB0, 0xAC, 0x89, 0x91, 0x3E, 0x5E, 0x76, 0xF3, 0xAC, 0x2E,
    0x2E, 0xF1, 0xB1, 0x8C, 0x5A, 0x9C, 0xAC, 0x2C, 0x28, 0x28, 0x28, 0x9C, 0xAC, 0x30, 0x18, 0xA8,
    0x98, 0x81, 0x28, 0x34, 0x3C, 0x97, 0x24, 0xA7, 0x28, 0x34, 0x3C, 0x9C, 0x24, 0xF2, 0xB0, 0x89,
    0xAC, 0x91, 0x2C, 0x4C, 0x6C, 0x8A, 0x9B, 0x2D, 0xD9, 0xD8, 0xD8, 0x51, 0xD9, 0xD8, 0xD8, 0x79,

    0xD9, 0xD8, 0xD8, 0xF1, 0x9E, 0x88, 0xA3, 0x31, 0xDA, 0xD8, 0xD8, 0x91, 0x2D, 0xD9, 0x28, 0xD8,
    0x4D, 0xD9, 0x48, 0xD8, 0x6D, 0xD9, 0x68, 0xD8, 0xB1, 0x83, 0x93, 0x35, 0x3D, 0x80, 0x25, 0xDA,
    0xD8, 0xD8, 0x85, 0x69, 0xDA, 0xD8, 0xD8, 0xB4, 0x93, 0x81, 0xA3, 0x28, 0x34, 0x3C, 0xF3, 0xAB,
    0x8B, 0xF8, 0xA3, 0x91, 0xB6, 0x09, 0xB4, 0xD9, 0xAB, 0xDE, 0xFA, 0xB0, 0x87, 0x9C, 0xB9, 0xA3,
    0xDD, 0xF1, 0xA3, 0xA3, 0xA3, 0xA3, 0x95, 0xF1, 0xA3, 0xA3, 0xA3, 0x9D, 0xF1, 0xA3, 0xA3, 0xA3,
    0xA3, 0xF2, 0xA3, 0xB4, 0x90, 0x80, 0xF2, 0xA3, 0xA3, 0xA3, 0xA3, 0xA3, 0xA3, 0xA3, 0xA3, 0xA3,
    0xA3, 0xB2, 0xA3, 0xA3, 0xA3, 0xA3, 0xA3, 0xA3, 0xB0, 0x87, 0xB5, 0x99, 0xF1, 0xA3, 0xA3, 0xA3,
    0x98, 0xF1, 0xA3, 0xA3, 0xA3, 0xA3, 0x97, 0xA3, 0xA3, 0xA3, 0xA3, 0xF3, 0x9B, 0xA3, 0xA3, 0xDC,
    0xB9, 0xA7, 0xF1, 0x26, 0x26, 0x26, 0xD8, 0xD8, 0xFF
]:
    fout.write(bytes([b]))
fout.close()



MemoryError: memory allocation failed, allocating 7728 bytes


In [102]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [22]:
from ubinascii import unhexlify as u
f = open("dmpMemory.bin", "wb")
w = f.write
w(u('FB00003E000B00360001000200030000'))
w(u('00650054FFEF0000FA80000B12820001'))
w(u('00020000000000000000000000000000'))
w(u('00280000FFFF4581FFFFFA7200000000'))
w(u('000003E80000000100017FFFFFFE8001'))
w(u('001B0000000000000000000000000000'))
w(u('003E03304000000002CAE3093E800000'))
w(u('20000000000000004000000060000000'))
w(u('41FF000000000B2A0000165500002182'))
w(u('FD872650FD800000001F000000058000'))
w(u('00000000000100000002000000030000'))
w(u('400000000000046F0002653200005EC0'))
w(u('40000000000000000000000000000000'))
w(u('FB8C6F5DFD5D08D9007C733B006C12CC'))
w(u('3200139D3200D0D632000800400001F4'))
w(u('FFE68079020000000000D0D600002710'))

w(u('FB000000400000000000000000000000'))
w(u('00000001000000000000010001000000'))
w(u('0000FA36FFBC308E0005FBF0FFD95BC8'))
w(u('FFD09ABE000010A9FFF41EB200CEBBF7'))
w(u('0000000100000004000200020200000C'))
w(u('FFC280000001800000CF800040000000'))
w(u('00000001000000000000060000000014'))
w(u('00000000000000000000000000000000'))
w(u('00000000000000000000000000000000'))
w(u('00000000000000000000000000000000'))
w(u('00000000033F68B6793528BCC67ED16C'))
w(u('80000000400000000000B26A00000000'))
w(u('000000000000000000003FF000000030'))
w(u('00000000000000000000000000000000'))
w(u('00000000000000000000000000000000'))
w(u('0000254D002F706D000005AE000C02D0'))

w(u('0000000000650054FFEF000000000000'))
w(u('000001000044000000000C0000000100'))
w(u('000000000065000000540000FFEF0000'))
w(u('00000000000000000000000000000000'))
w(u('40000000000000000000000000000000'))
w(u('40000000000000000000000000000000'))
w(u('00000001000000020000000000000000'))
w(u('00000000000000000000000000000000'))
w(u('00000000000000000000000000000000'))
w(u('001B0000000000000000000000000000'))
w(u('00000000000000000000000000000000'))
w(u('00000000000000000000000040000000'))
w(u('001B0000000000000000000000000000'))
w(u('00000000000000000000000000000000'))
w(u('00000000000000000000000000000000'))
w(u('00000000000000000000000000000000'))

w(u('D8DCBAA2F1DEB2B8B4A88191F74A907F'))
w(u('916AF3F9DBA8F9B0BAA080F2CE81F3C2'))
w(u('F1C1F2C3F3CCA2B280F1C6D880BAA7DF'))
w(u('DFDFF2A7C3CBC5B6F087A2942448703C'))
w(u('95406834589B78A2F183922D557DD8B1'))
w(u('B4B8A1D09180F270F370F27C80A8F101'))
w(u('B09887D943D886C988BAA1F20EB89780'))
w(u('F1A9DFDFDFAADFDFDFF2AAC5CDC7A90C'))
w(u('C92C97979797F1A989264666B0B4BA80'))
w(u('ACDEF2CAF1B28C02A9B69800890E161E'))
w(u('B8A9B4992C547CB08AA896365676F1B9'))
w(u('AFB4B083C0B8A89711B18F98B9AFF024'))
w(u('0844106418F1A329557DAF83B593AFF0'))
w(u('002850F1A3869F61A6DADEDFD9FAA386'))
w(u('96DB31A6D9F8DFBAA68FC2C5C7B28CC1'))
w(u('B8A2DFDFDFA3DFDFDFD8D8F1B8A8B286'))

w(u('B4980D355DB8AA98B0872D353DB2B6BA'))
w(u('AF8C96198F9FA70E161EB49AB8AA872C'))
w(u('547CB9A3DEDFDFA3B180F2C4CDC9F1B8'))
w(u('A9B499830D355D89B9A32D557DB593A3'))
w(u('0E161EA92C547CB8B4B0F19783A81184'))
w(u('A50998A383F0DA240844106418D8F1A5'))
w(u('29557DA58595021A2E3A565A4048F9F3'))
w(u('A3D9F8F098832408441064189782A8F1'))
w(u('11F098A2240844106418DAF3DED883A5'))
w(u('9401D9A302F1A2C3C5C7D8F18492A24D'))
w(u('DA2AD84869D92AD86855DA32D85071D9'))
w(u('32D8705DDA3AD85879D93AD87893A34D'))
w(u('DA2AD84869D92AD86855DA32D85071D9'))
w(u('32D8705DDA3AD85879D93AD878A88A9A'))
w(u('F02850789EF38818F19F1D98A8D908D8'))
w(u('C89F129EF315A8DA1210D8F1AFC89787'))

w(u('34B5B994A421F3D922D8F22DF3D92AD8'))
w(u('F235F3D932D881A4606061D961D86C68'))
w(u('69D969D8747071D971D8B1A384193D5D'))
w(u('A3831A3E5E931030811011B8B0AF8F94'))
w(u('F2DA3ED8B49AA88729DAF8D8879A35DA'))
w(u('F8D8879A3DDAF8D8B1B9A49885022E56'))
w(u('A581000C14A397B08AF12DD928D84DD9'))
w(u('48D86DD968D8B1840DDA0ED8A32983DA'))
w(u('2C0ED8A3844983DA2C4C0ED8B8B0A88A'))
w(u('9AF520AADADFD8A840AAD0DADED8A860'))
w(u('AADAD0DFD8F19786A8319B069907AB97'))
w(u('28889BF00C201440B8B0B4A88C9CF004'))
w(u('2851791D301438B282ABD0982C505078'))
w(u('789BF11AB0F08A9CA82951798B295179'))
w(u('8A2470598B2058718A4469388B394068'))
w(u('8A6448318B304960A588200971584468'))

w(u('113964493019F1AC002C547CF08CA804'))
w(u('285078F1889726A85998AC8C02264666'))
w(u('F0899CA8295179247059446938644831'))
w(u('A98809205970AB11384069A819314860'))
w(u('8CA83C415C207C00F187981986A86E76'))
w(u('7EA999882D557D9EB9A38A228A6E8A56'))
w(u('8A5E9FB183062646660E2E4E6E9DB8AD'))
w(u('002C547CF2B18CB499B9A32D557D8191'))
w(u('AC38AD3AB58391AC2DD928D84DD948D8'))
w(u('6DD968D88C9DAE29D904AED851D904AE'))
w(u('D879D904D881F39DAD008DAE1981ADD9'))
w(u('01D8F2AEDA26D88E912983A7D9ADADAD'))
w(u('ADF32AD8D8F1B0AC89913E5E76F3AC2E'))
w(u('2EF1B18C5A9CAC2C2828289CAC3018A8'))
w(u('988128343C9724A728343C9C24F2B089'))
w(u('AC912C4C6C8A9B2DD9D8D851D9D8D879'))

w(u('D9D8D8F19E88A331DAD8D8912DD928D8'))
w(u('4DD948D86DD968D8B18393353D8025DA'))
w(u('D8D88569DAD8D8B49381A328343CF3AB'))
w(u('8BF8A391B609B4D9ABDEFAB0879CB9A3'))
w(u('DDF1A3A3A3A395F1A3A3A39DF1A3A3A3'))
w(u('A3F2A3B49080F2A3A3A3A3A3A3A3A3A3'))
w(u('A3B2A3A3A3A3A3A3B087B599F1A3A3A3'))
w(u('98F1A3A3A3A397A3A3A3A3F39BA3A3DC'))
w(u('B9A7F1262626D8D8FF'))
f.close()

Sent 1929 bytes in 65 chunks to dmpMemory.bin.


In [19]:
%ls


Listing directory '/'.
        0    binary
      139    boot.py
     1929    dmpMemory.bin


In [33]:
%sendtofile dmpConfig.txt

//  BANK    OFFSET  LENGTH  [DATA]
    0x02,   0xEC,   0x04,   0x00, 0x47, 0x7D, 0x1A,   // ?
    0x03,   0x82,   0x03,   0x4C, 0xCD, 0x6C,         // FCFG_1 inv_set_gyro_calibration
    0x03,   0xB2,   0x03,   0x36, 0x56, 0x76,         // FCFG_3 inv_set_gyro_calibration
    0x00,   0x68,   0x04,   0x02, 0xCA, 0xE3, 0x09,   // D_0_104 inv_set_gyro_calibration
    0x01,   0x0C,   0x04,   0x00, 0x00, 0x00, 0x00,   // D_1_152 inv_set_accel_calibration
    0x03,   0x86,   0x03,   0x0C, 0xC9, 0x2C,         // FCFG_2 inv_set_accel_calibration
    0x03,   0x90,   0x03,   0x26, 0x46, 0x66,         //   (continued)...FCFG_2 inv_set_accel_calibration
    0x00,   0x6C,   0x02,   0x40, 0x00,               // D_0_108 inv_set_accel_calibration

    0x02,   0x40,   0x04,   0x00, 0x00, 0x00, 0x00,   // CPASS_MTX_00 inv_set_compass_calibration
    0x02,   0x44,   0x04,   0x40, 0x00, 0x00, 0x00,   // CPASS_MTX_01
    0x02,   0x48,   0x04,   0x00, 0x00, 0x00, 0x00,   // CPASS_MTX_02
    0x02,   0x4C,   0x04,   0x40, 0x00, 0x00, 0x00,   // CPASS_MTX_10
    0x02,   0x50,   0x04,   0x00, 0x00, 0x00, 0x00,   // CPASS_MTX_11
    0x02,   0x54,   0x04,   0x00, 0x00, 0x00, 0x00,   // CPASS_MTX_12
    0x02,   0x58,   0x04,   0x00, 0x00, 0x00, 0x00,   // CPASS_MTX_20
    0x02,   0x5C,   0x04,   0x00, 0x00, 0x00, 0x00,   // CPASS_MTX_21
    0x02,   0xBC,   0x04,   0xC0, 0x00, 0x00, 0x00,   // CPASS_MTX_22

    0x01,   0xEC,   0x04,   0x00, 0x00, 0x40, 0x00,   // D_1_236 inv_apply_endian_accel
    0x03,   0x86,   0x06,   0x0C, 0xC9, 0x2C, 0x97, 0x97, 0x97, // FCFG_2 inv_set_mpu_sensors
    0x04,   0x22,   0x03,   0x0D, 0x35, 0x5D,         // CFG_MOTION_BIAS inv_turn_on_bias_from_no_motion
    0x00,   0xA3,   0x01,   0x00,                     // ?
    0x04,   0x29,   0x04,   0x87, 0x2D, 0x35, 0x3D,   // FCFG_5 inv_set_bias_update
    0x07,   0x62,   0x05,   0xF1, 0x20, 0x28, 0x30, 0x38, // CFG_8 inv_send_quaternion
    0x07,   0x9F,   0x01,   0x30,                     // CFG_16 inv_set_footer
    0x07,   0x67,   0x01,   0x9A,                     // CFG_GYRO_SOURCE inv_send_gyro
    0x07,   0x68,   0x04,   0xF1, 0x28, 0x30, 0x38,   // CFG_9 inv_send_gyro -> inv_construct3_fifo
    0x07,   0x62,   0x05,   0xF1, 0x20, 0x28, 0x30, 0x38, // ?
    0x02,   0x0C,   0x04,   0x00, 0x00, 0x00, 0x00,   // ?
    0x07,   0x83,   0x06,   0xC2, 0xCA, 0xC4, 0xA3, 0xA3, 0xA3, // ?
                 // SPECIAL 0x01 = enable interrupts
    0x00,   0x00,   0x00,   0x01, // SET INT_ENABLE, SPECIAL INSTRUCTION
    0x07,   0xA7,   0x01,   0xFE,                     // ?
    0x07,   0x62,   0x05,   0xF1, 0x20, 0x28, 0x30, 0x38, // ?
    0x07,   0x67,   0x01,   0x9A,                     // ?
    0x07,   0x68,   0x04,   0xF1, 0x28, 0x30, 0x38,   // CFG_12 inv_send_accel -> inv_construct3_fifo
    0x07,   0x8D,   0x04,   0xF1, 0x28, 0x30, 0x38,   // ??? CFG_12 inv_send_mag -> inv_construct3_fifo
    0x02,   0x16,   0x02,   0x00, 0x01, // MPU6050_DMP_FIFO_RATE_DIVISOR // D_0_22 inv_set_fifo_rate

    // This very last 0x03 WAS a 0x09, which drops the FIFO rate down to 20 Hz. 0x07 is 25 Hz,
    // 0x01 is 100Hz. Going faster than 100Hz (0x00=200Hz) tends to result in very noisy data.
    // DMP output frequency is calculated easily using this equation: (200Hz / (1 + value))

    // It is important to make sure the host processor can keep up with reading and processing
    // the FIFO output at the desired rate. Handling FIFO overflow cleanly is also a good idea.

Sent 47 lines (3462 bytes) to dmpConfig.txt.


In [126]:
os.remove("main.py")

In [36]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


[2, 236, 4, 0, 71, 125, 26]
[3, 130, 3, 76, 205, 108]
[3, 178, 3, 54, 86, 118]
[0, 104, 4, 2, 202, 227, 9]
[1, 12, 4, 0, 0, 0, 0]
[3, 134, 3, 12, 201, 44]
[3, 144, 3, 38, 70, 102]
[0, 108, 2, 64, 0]
[2, 64, 4, 0, 0, 0, 0]
[2, 68, 4, 64, 0, 0, 0]
[2, 72, 4, 0, 0, 0, 0]
[2, 76, 4, 64, 0, 0, 0]
[2, 80, 4, 0, 0, 0, 0]
[2, 84, 4, 0, 0, 0, 0]
[2, 88, 4, 0, 0, 0, 0]
[2, 92, 4, 0, 0, 0, 0]
[2, 188, 4, 192, 0, 0, 0]
[1, 236, 4, 0, 0, 64, 0]
[3, 134, 6, 12, 201, 44, 151, 151, 151]
[4, 34, 3, 13, 53, 93]
[0, 163, 1, 0]
[4, 41, 4, 135, 45, 53, 61]
[7, 98, 5, 241, 32, 40, 48, 56]
[7, 159, 1, 48]
[7, 103, 1, 154]
[7, 104, 4, 241, 40, 48, 56]
[7, 98, 5, 241, 32, 40, 48, 56]
[2, 12, 4, 0, 0, 0, 0]
[7, 131, 6, 194, 202, 196, 163, 163, 163]
[0, 0, 0, 1]
[7, 167, 1, 254]
[7, 98, 5, 241, 32, 40, 48, 56]
[7, 103, 1, 154]
[7, 104, 4, 241, 40, 48, 56]
[7, 141, 4, 241, 40, 48, 56]
[2, 22, 2, 0, 1]
